In [95]:
# Opzionale, scarica le sprite dei Pokémon per il gioco
!python ./scripts/sprite_downloader.py

Starting sprite download...
Not found (404): https://play.pokemonshowdown.com/sprites/ani/deoxysattack-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/wormadam-plant-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/wormadam-plant-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/wormadam-sandy-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/mr-mime.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/deoxys-normal-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/primal-groudon.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/deoxys-defense-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/primal-kyogre.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/wormadam-sandy-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/deoxys-speed-forme.gif
Not found (404): http

In [109]:
# Librerie base
import pandas as pd
import numpy as np

# Visualizzazione
import matplotlib
from matplotlib.colors import to_rgb
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# Machine Learning
import joblib
from sklearn.preprocessing import FunctionTransformer

# Management di file
import os
import io

# Flask
from flask import Flask, send_file, abort, request, jsonify, render_template

## Configurazione di base

In [97]:
# Configurazione di matplotlib
matplotlib.use('Agg')
FONT_PATH: str = os.path.join(os.path.dirname('./'), 'webpage', 'static', 'font', 'PokeClassic.ttf')
font_properties: fm.FontProperties = fm.FontProperties(fname=FONT_PATH)

In [98]:
# Definizione dei percorsi e dei parametri globali
DATASET_BASE_PATH: str = './datasets'
COMBATS_PATH: str = f"{DATASET_BASE_PATH}/combats.csv"
TYPE_CHART_PATH: str = f"{DATASET_BASE_PATH}/type_chart.csv"
POKEMON_PATH: str = f"{DATASET_BASE_PATH}/pokemon.csv"
MODEL_DIR: str = './models'

## Caricamento dei dati

In [ ]:
# Caricamento dei DataFrame
df_type_chart = pd.read_csv(TYPE_CHART_PATH).fillna("None")
df_pokemon = pd.read_csv(POKEMON_PATH, index_col="#").fillna("None")

In [ ]:
# Caricamento del trasformatore di funzioni
from utils import function_transformer
data_transformer: FunctionTransformer = FunctionTransformer(func=function_transformer, kw_args={'dataframe_type_chart': df_type_chart}).set_output(transform="pandas")

## Funzioni di utility

In [ ]:
# Definizione dei colori per i tipi di Pokémon
type_colors: dict[str, str] = {
   "Normal": "#A8A878",
   "Fire": "#F08030",
   "Water": "#6890F0",
   "Grass": "#78C850",
   "Electric": "#F8D030",
   "Ice": "#98D8D8",
   "Fighting": "#C03028",
   "Poison": "#A040A0",
   "Ground": "#E0C068",
   "Flying": "#A890F0",
   "Psychic": "#F85888",
   "Bug": "#A8B820",
   "Rock": "#B8A038",
   "Ghost": "#705898",
   "Dragon": "#7038F8",
   "Dark": "#705848",
   "Steel": "#B8B8D0",
   "Fairy": "#EE99AC"
}

In [ ]:
# Funzione per interpolare i colori tra due valori
def interpolate_color(c1, c2, t):
   c1 = np.array(to_rgb(c1))
   c2 = np.array(to_rgb(c2))
   return c1 * (1 - t) + c2 * t

In [ ]:
# Funzione per creare un grafico radar per le statistiche di un Pokémon
def generate_pokemon_radial_graph(name):
   # Otteniamo le statistiche del Pokémon
   stats = df_pokemon[df_pokemon["Name"] == name].iloc[0].to_dict()
   # Otteniamo il tipo del Pokémon per colorare il grafico
   type_first = stats['Type 1']
   type_second = stats['Type 2']
   if type_second == 'None' or type_second == '' or type_second is None:
      type_second = type_first
   color_center = type_colors[type_first]
   color_edge = type_colors[type_second]
   # Creazione delle etichette e dei valori per il grafico
   labels = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
   values = [stats[label] for label in labels]
   # Aggiungiamo il primo valore alla fine per chiudere il grafico
   angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
   values += values[:1]
   angles += angles[:1]
   # Creazione del grafico radar
   fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True), facecolor='none')
   ax.set_facecolor('none')
   # Normalizza i valori delle statistiche
   max_stat = max(values)
   norm_values = np.array(values) / max_stat
   # Traccia le aree del grafico con un gradiente
   steps = 40
   for i in range(steps):
      t = i / (steps - 1)
      layer_values = norm_values * t * max_stat
      layer_color = interpolate_color(color_center, color_edge, t)
      ax.fill(angles, layer_values, color=layer_color, alpha=0.05, zorder=steps - i)
   # Traccia il bordo del grafico
   ax.plot(angles, values, color=color_edge, linewidth=2, zorder=steps + 1)
   # Impostazione delle etichette degli assi
   ax.set_xticks(angles[:-1])
   ax.set_xticklabels(labels, color='white', fontsize=11)
   # Impostazione del colore e della dimensione dei tick
   for tick_label in ax.get_xticklabels():
      tick_label.set_color('white')
      tick_label.set_fontsize(11)
      tick_label.set_fontproperties(font_properties)
   # Titolo
   title_text = ax.set_title(f"{name}", va='bottom', color=color_center)
   title_text.set_fontsize(17)
   title_text.set_fontproperties(font_properties)
   # Setup assi radiali
   ax.set_rlabel_position(-90)
   ax.set_ylim(0, max_stat + 10)
   ax.grid(color='white', linestyle='--', linewidth=0.8, alpha=0.5)
   ax.spines['polar'].set_color('white')
   ax.grid(True)
   # Stile per le etichette radiali
   for label in ax.get_yticklabels():
      label.set_color('white')
      label.set_fontsize(7)
      label.set_fontproperties(font_properties)
   # Salvataggio del grafico in un buffer di memoria
   buf = io.BytesIO()
   fig.savefig(buf, format='png', transparent=True, bbox_inches='tight')
   plt.close(fig)
   buf.seek(0)
   return buf.getvalue()


## Pagina web

In [144]:
app: Flask = Flask(__name__, template_folder='webpage', static_folder="webpage/static")

models = os.listdir(MODEL_DIR)
model = joblib.load(os.path.join(MODEL_DIR, models[0]))

@app.route('/', methods=['GET'])
def one_v_one_battle():
    return render_template('1v1_battle.html', models=models, pokemon_choices=df_pokemon['Name'])

@app.route('/change_model', methods=['POST'])
def change_model():
    # Controlla se il corpo della richiesta è in formato JSON
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    # Controlla se il modello specificato esiste
    model_filename: str = data.get('model_file')
    if not model_filename or model_filename not in models:
        return jsonify({"error": "Model file not found."}), 404
    # Cambia il modello di machine learning in uso
    global model
    model = joblib.load(os.path.join(MODEL_DIR, model_filename))
    return jsonify({"message": f"Model changed to {model_filename}."}), 200

@app.route('/pokemon/<name>/radial_stats.png')
def pokemon_radial(name):
    if name not in df_pokemon["Name"].values:
        abort(404, description=f"Pokémon '{name}' non trovato")
    img_bytes = generate_pokemon_radial_graph(name)
    return send_file(
        io.BytesIO(img_bytes),
        mimetype="image/png",
        as_attachment=False,
        download_name=f"{name}_radial_stats.png"
    )

@app.route('/precdict_battle', methods=['POST'])
def predict_battle():
    # Controlla se il corpo della richiesta è in formato JSON
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    # Controlla se i Pokémon sono specificati
    p1: str = data.get("pokemon_first")
    p2: str = data.get("pokemon_second")
    if not p1 or not p2:
        return jsonify({"error": "Both Pokémon must be specified."}), 400
    # Prepara i dati per la previsione
    first_data = df_pokemon.loc[df_pokemon['Name'] == p1].add_suffix(suffix='_F')
    second_data = df_pokemon.loc[df_pokemon['Name'] == p2].add_suffix(suffix='_S')
    # Predizione del vincitore
    winner_id = model.predict(first_data.merge(second_data, how='cross'))
    winner = p1 if winner_id == 0 else p2
    winner += " (Left)" if winner_id == 0 else " (Right)"
    confidence = np.max(model.predict_proba(first_data.merge(second_data, how='cross')), axis=1)[0] * 100
    return jsonify({
        "winner": winner,
        "confidence": float(confidence)
    })

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
